# Pycoornet Example



In [ ]:
# Install pycoornet
!pip install -U pycoornet -q

## Import libraries

In [ ]:
from pycoornet.crowdtangle import CrowdTangle
from pycoornet.shared import Shared
from pycoornet.statistics import Statistics
import networkx as nx
import numpy as np
import pandas as pd
import logging

## Get Links

In [ ]:
# Read data links
links_df = pd.read_csv('sample_source_links.csv')

In [ ]:
links_df.info()

## Get Crowdtangle information

In [ ]:
# start logging info
logging.basicConfig(level=logging.INFO, filename="pycoornet.log")

In [ ]:
# init Crowtangle with api key
crowd_tangle = CrowdTangle("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

In [ ]:
#Get links from crowdtangle
ct_df = crowd_tangle.get_shares(urls=links_df, url_column='clean_url', date_column='date',clean_urls=True, platforms='facebook', sleep_time=30)

In [ ]:
shared = Shared(ct_df)
crowtangle_shares_df, shares_graph, q = shared.coord_shares(clean_urls=True)

## Build Gephi File

In [ ]:
for node in shares_graph.nodes(data=True):
    node[1]['label']=node[1]['account_name']
nx.write_gexf(shares_graph, "out/shares_network.gexf")

### Plot Network

In [ ]:
import logging
import math
import matplotlib as mpl
import matplotlib.pyplot as plt

#install plotly for the 3d network interactive plot
import plotly.graph_objects as go

from pycoornet.utils import Utils

In [ ]:
colors = [node[1]['cluster'] for node in shares_graph.nodes(data=True)]
low, *_, high = sorted(colors)
norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
mapper = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.coolwarm)

In [ ]:
labels = {node[0]:node[1]['account_name'][0:15] for node in shares_graph.nodes(data=True)}

In [ ]:
strengths = [node[1]['strength'] for node in shares_graph.nodes(data=True)]

In [ ]:
nx_layout = nx.spring_layout(shares_graph,dim=3, seed=18, k=5/math.sqrt(shares_graph.order()))

In [ ]:
nx_2d_layout = nx.spring_layout(shares_graph,iterations=70, k=7/math.sqrt(shares_graph.order()), seed=18)
plt.figure(figsize=(10,10))
nx.draw(shares_graph,
        pos=nx_2d_layout, 
        cmap=plt.get_cmap('viridis'), 
        node_color=colors,
        with_labels=False)
fig = nx.draw_networkx_labels(shares_graph, pos=nx_2d_layout, labels=labels)

In [ ]:
x_nodes = [value[0] for key, value in nx_layout.items()]
y_nodes = [value[1] for key, value in nx_layout.items()]
z_nodes = [value[2] for key, value in nx_layout.items()]

In [ ]:
edge_list = shares_graph.edges

In [ ]:
norm = np.linalg.norm(strength)
normal_strength = strength/norm

In [ ]:
#we  need to create lists that contain the starting and ending coordinates of each edge.
x_edges=[]
y_edges=[]
z_edges=[]

#need to fill these with all of the coordiates
for edge in edge_list:
    #format: [beginning,ending,None]
    x_coords = [nx_layout[edge[0]][0],nx_layout[edge[1]][0],None]
    x_edges += x_coords

    y_coords = [nx_layout[edge[0]][1],nx_layout[edge[1]][1],None]
    y_edges += y_coords

    z_coords = [nx_layout[edge[0]][2],nx_layout[edge[1]][2],None]
    z_edges += z_coords

In [ ]:
#create a trace for the edges
trace_edges = go.Scatter3d(x=x_edges,
                        y=y_edges,
                        z=z_edges,
                        mode='lines',
                        line=dict(color='rgb(125,125,125)', width=1),
                        hoverinfo='none')

In [ ]:
#create a trace for the nodes
trace_nodes = go.Scatter3d(x=x_nodes,
                         y=y_nodes,
                        z=z_nodes,
                        mode='markers',
                        marker=dict(symbol='circle',
                                    size=normal_strength * 150,
                                    color=colors, #color the nodes according to their community
                                    colorscale='Viridis',
                                    line=dict(color='rgb(50,50,50)', width=0.5)),
                        text=[node[1]['account_name'] for node in shares_graph.nodes(data=True)],
                        hoverinfo='text')

In [ ]:
#we need to set the axis for the plot 
axis = dict(showbackground=False,
            showline=False,
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            title='')

In [ ]:
#also need to create the layout for our plot
layout = go.Layout(title="Network Results",
                width=650,
                height=625,
                showlegend=False,
                scene=dict(xaxis=dict(axis),
                        yaxis=dict(axis),
                        zaxis=dict(axis),
                        ),
                margin=dict(t=100),
                hovermode='closest')

In [ ]:
#Include the traces we want to plot and create a figure
data = [trace_edges, trace_nodes]
fig = go.Figure(data=data, layout=layout)
fig.show()

## Get General Statistical Information

In [ ]:
componet_summary_df = Statistics.component_summary(crowtangle_shares_df, shares_graph)

In [ ]:
componet_summary_df

In [ ]:
top_urls_df = Statistics.get_top_coord_urls(crowtangle_shares_df, shares_graph)

In [ ]:
top_urls_df

## Acknowledgements
CooRnet has been developed as part of the project [Social Media Behaviour](https://upb-ss1.github.io/) research project activities.

The project is supported by a the Social Media and Democracy Research Grants from Social Science Research Council (SSRC). Data and tools provided by Facebook.